## Facts & Dimension 

In [0]:
Create database sales_new

In [0]:
CREATE OR REPLACE TABLE sales_new.Orders (
    OrderID INT,
    OrderDate DATE,
    CustomerID INT,
    CustomerName VARCHAR(100),
    CustomerEmail VARCHAR(100),
    ProductID INT,
    ProductName VARCHAR(100),
    ProductCategory VARCHAR(50),
    RegionID INT,
    RegionName VARCHAR(50),
    Country VARCHAR(50),
    Quantity INT,
    UnitPrice DECIMAL(10,2),
    TotalAmount DECIMAL(10,2))

In [0]:
INSERT INTO sales_new.Orders (OrderID, OrderDate, CustomerID, CustomerName, CustomerEmail, ProductID, ProductName, ProductCategory, RegionID, RegionName, Country, Quantity, UnitPrice, TotalAmount) 
VALUES 
(1, '2024-02-01', 101, 'Alice Johnson', 'alice@example.com', 201, 'Laptop', 'Electronics', 301, 'North America', 'USA', 2, 800.00, 1600.00),
(2, '2024-02-02', 102, 'Bob Smith', 'bob@example.com', 202, 'Smartphone', 'Electronics', 302, 'Europe', 'Germany', 1, 500.00, 500.00),
(3, '2024-02-03', 103, 'Charlie Brown', 'charlie@example.com', 203, 'Tablet', 'Electronics', 303, 'Asia', 'India', 3, 300.00, 900.00),
(4, '2024-02-04', 101, 'Alice Johnson', 'alice@example.com', 204, 'Headphones', 'Accessories', 301, 'North America', 'USA', 1, 150.00, 150.00),
(5, '2024-02-05', 104, 'David Lee', 'david@example.com', 205, 'Gaming Console', 'Electronics', 302, 'Europe', 'France', 1, 400.00, 400.00),
(6, '2024-02-06', 102, 'Bob Smith', 'bob@example.com', 206, 'Smartwatch', 'Electronics', 303, 'Asia', 'China', 2, 200.00, 400.00),
(7, '2024-02-07', 105, 'Eve Adams', 'eve@example.com', 201, 'Laptop', 'Electronics', 301, 'North America', 'Canada', 1, 800.00, 800.00),
(8, '2024-02-08', 106, 'Frank Miller', 'frank@example.com', 207, 'Monitor', 'Accessories', 302, 'Europe', 'Italy', 2, 250.00, 500.00),
(9, '2024-02-09', 107, 'Grace White', 'grace@example.com', 208, 'Keyboard', 'Accessories', 303, 'Asia', 'Japan', 3, 100.00, 300.00),
(10, '2024-02-10', 104, 'David Lee', 'david@example.com', 209, 'Mouse', 'Accessories', 301, 'North America', 'USA', 1, 50.00, 50.00);

In [0]:
Select * from sales_new.Orders

### Data Warehousing

In [0]:
create database orders_dwh

Staging layer

In [0]:
Create or Replace table orders_dwh.stg_sales
As
select * from sales_new.orders

Transformation Layer 

In [0]:
CREATE VIEW orders_dwh.trans_sales
AS
SELECT * FROM orders_dwh.stg_sales WHERE Quantity IS NOT NULL 

In [0]:
SELECT * FROM orders_dwh.trans_sales

### Core Layer

#### Customer Dimension

In [0]:
CREATE OR REPLACE TABLE orders_dwh.DimCustomers 
(
  CustomerID INT,
  CustomerName STRING,
  CustomerEmail STRING,
  DimCustomersKey INT
)

In [0]:
-- Creating a view to get the stream of data 
Create or replace view orders_dwh.view_cust
AS 
Select t.*,row_number() over(order by t.CustomerID) as DimCustomersKey
From
(Select 
DISTINCT (CustomerID) AS CustomerID,
CustomerName, 
CustomerEmail 
FROM orders_dwh.trans_sales)t

In [0]:
Select * from orders_dwh.view_cust

In [0]:
INSERT into orders_dwh.DimCustomers 
Select * from orders_dwh.view_cust

#### 